In [1]:
import re
import nltk
import string
import numpy as np
import pandas as pd
from nltk import pos_tag
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nitro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Nitro\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Nitro\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\Nitro\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

## Import Data

In [2]:
data=pd.read_csv("dialogs_expanded.csv")
data

,Unnamed: 0,question,answer,question_as_int,answer_as_int,question_len,answer_len
0,1,"Well, I thought we'd start with pronunciation,...",Not the hacking and gagging and spitting part....,"[54, 67, 74, 74, 12, 1, 40, 1, 82, 70, 77, 83,...","[45, 77, 82, 1, 82, 70, 67, 1, 70, 63, 65, 73,...",71,55
1,2,Not the hacking and gagging and spitting part....,Okay... then how 'bout we try out some French ...,"[45, 77, 82, 1, 82, 70, 67, 1, 70, 63, 65, 73,...","[46, 73, 63, 87, 14, 14, 14, 1, 82, 70, 67, 76...",55,73
2,3,You're asking me out. That's so cute. What's ...,Forget it.,"[56, 77, 83, 8, 80, 67, 1, 63, 81, 73, 71, 76,...","[37, 77, 80, 69, 67, 82, 1, 71, 82, 14]",62,10
3,4,"No, no, it's my fault -- we didn't have a prop...",Cameron.,"[45, 77, 12, 1, 76, 77, 12, 1, 71, 82, 8, 81, ...","[34, 63, 75, 67, 80, 77, 76, 14]",65,8
4,9,"Gosh, if only we could find Kat a boyfriend...",Let me see what I can do.,"[38, 77, 81, 70, 12, 1, 71, 68, 1, 77, 76, 74,...","[43, 67, 82, 1, 75, 67, 1, 81, 67, 67, 1, 85, ...",46,25
...,...,...,...,...,...,...,...
139404,221608,Well that one. The one who keeps looking at me.,ft could be you flatter yourself CoghilL It's ...,"[54, 67, 74, 74, 1, 82, 70, 63, 82, 1, 77, 76,...","[68, 82, 1, 65, 77, 83, 74, 66, 1, 64, 67, 1, ...",47,59
139405,221609,Choose your targets men. That's right Watch th...,Keep steady. You're the best shots of the Twen...,"[34, 70, 77, 77, 81, 67, 1, 87, 77, 83, 80, 1,...","[42, 67, 67, 78, 1, 81, 82, 67, 63, 66, 87, 14...",61,85
139406,221610,Colonel Durnford... William Vereker. I hear yo...,"Good ones, yes, Mr Vereker. Gentlemen who can ...","[34, 77, 74, 77, 76, 67, 74, 1, 35, 83, 80, 76...","[38, 77, 77, 66, 1, 77, 76, 67, 81, 12, 1, 87,...",74,60
139407,221611,"Your orders, Mr Vereker?",I'm to take the Sikali with the main column to...,"[56, 77, 83, 80, 1, 77, 80, 66, 67, 80, 81, 12...","[40, 8, 75, 1, 82, 77, 1, 82, 63, 73, 67, 1, 8...",24,56


## Select Columns

In [3]:
df=data[['question','answer']]
df

,question,answer
0,"Well, I thought we'd start with pronunciation,...",Not the hacking and gagging and spitting part....
1,Not the hacking and gagging and spitting part....,Okay... then how 'bout we try out some French ...
2,You're asking me out. That's so cute. What's ...,Forget it.
3,"No, no, it's my fault -- we didn't have a prop...",Cameron.
4,"Gosh, if only we could find Kat a boyfriend...",Let me see what I can do.
...,...,...
139404,Well that one. The one who keeps looking at me.,ft could be you flatter yourself CoghilL It's ...
139405,Choose your targets men. That's right Watch th...,Keep steady. You're the best shots of the Twen...
139406,Colonel Durnford... William Vereker. I hear yo...,"Good ones, yes, Mr Vereker. Gentlemen who can ..."
139407,"Your orders, Mr Vereker?",I'm to take the Sikali with the main column to...


## Check Nulls Values 

In [4]:
df.isnull().sum()

question    0
answer      0
dtype: int64

## check and drop Duplicated values



In [5]:
print(df.duplicated().sum())

710


In [6]:
df.drop_duplicates(inplace=True)

C:\Users\Nitro\AppData\Local\Temp\ipykernel_18512\3006716147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [7]:
print(df.duplicated().sum())

0


## Check the Number of Emojis 

In [8]:
emoji_pattern = re.compile(
    "["
    u"\U0001F600-\U0001F64F"
    u"\U0001F300-\U0001F5FF"
    u"\U0001F680-\U0001F6FF"
    u"\U0001F1E0-\U0001F1FF"
    u"\U00002700-\U000027BF"
    u"\U0001F900-\U0001F9FF"
    u"\U00002600-\U000026FF"
    "]+",

    flags=re.UNICODE,
)

def contains_emoji(text):
    return bool(emoji_pattern.search(text))

In [9]:
print(df["question"].apply(contains_emoji).sum())

0


In [10]:
df['question']

0         Well, I thought we'd start with pronunciation,...
1         Not the hacking and gagging and spitting part....
2         You're asking me out.  That's so cute. What's ...
3         No, no, it's my fault -- we didn't have a prop...
4            Gosh, if only we could find Kat a boyfriend...
                                ...                        
139404      Well that one. The one who keeps looking at me.
139405    Choose your targets men. That's right Watch th...
139406    Colonel Durnford... William Vereker. I hear yo...
139407                             Your orders, Mr Vereker?
139408    I'm to take the Sikali with the main column to...
Name: question, Length: 138699, dtype: object

In [11]:
df["answer"]

0         Not the hacking and gagging and spitting part....
1         Okay... then how 'bout we try out some French ...
2                                                Forget it.
3                                                  Cameron.
4                                 Let me see what I can do.
                                ...                        
139404    ft could be you flatter yourself CoghilL It's ...
139405    Keep steady. You're the best shots of the Twen...
139406    Good ones, yes, Mr Vereker. Gentlemen who can ...
139407    I'm to take the Sikali with the main column to...
139408    Lord Chelmsford seems to want me to stay back ...
Name: answer, Length: 138699, dtype: object

## ChatWords & Contractions

In [12]:
chat_words = {
    'AFAIK':'As Far As I Know',
    'AFK':'Away From Keyboard',
    'ASAP':'As Soon As Possible',
    "FYI": "For Your Information",
    "ASAP": "As Soon As Possible",
    "BRB": "Be Right Back",
    "BTW": "By The Way",
    "OMG": "Oh My God",
    "IMO": "In My Opinion",
    "LOL": "Laugh Out Loud",
    "TTYL": "Talk To You Later",
    "GTG": "Got To Go",
    "TTYT": "Talk To You Tomorrow",
    "IDK": "I Don't Know",
    "TMI": "Too Much Information",
    "IMHO": "In My Humble Opinion",
    "ICYMI": "In Case You Missed It",
    "AFAIK": "As Far As I Know",
    "BTW": "By The Way",
    "FAQ": "Frequently Asked Questions",
    "TGIF": "Thank God It's Friday",
    "FYA": "For Your Action",
    "ICYMI": "In Case You Missed It"
}

contractions = {
    "isn't": "is not",
    "aren't": "are not",
    "can't": "cannot",
    "couldn't": "could not",
    "don't": "do not",
    "doesn't": "does not",
    "didn't": "did not",
    "won't": "will not",
    "wouldn't": "would not",
    "shouldn't": "should not",
    "haven't": "have not",
    "hasn't": "has not",
    "hadn't": "had not",
    "i'm": "i am",
    "im":"i am",
    "i've": "i have",
    "i'd": "i had",
    "i'll": "i will",
    "you're": "you are",
    "you've": "you have",
    "you'd": "you would",
    "you'll": "you will",
    "he's": "he is",
    "he'd": "he would",
    "he'll": "he will",
    "she's": "she is",
    "she'd": "she would",
    "she'll": "she will",
    "it's": "it is",
    "it'd": "it would",
    "it'll": "it will",
    "we're": "we are",
    "we've": "we have",
    "we'd": "we would",
    "we'll": "we will",
    "they're": "they are",
    "they've": "they have",
    "they'd": "they would",
    "they'll": "they will",
    "that's": "that is",
    "thats": "that is",
    "there's": "there is",
    "there'd": "there would",
    "who's": "who is",
    "who'd": "who would",
    "who'll": "who will",
    "what's": "what is",
    "what're": "what are",
    "what'd": "what did",
    "where's": "where is",
    "when's": "when is",
    "why's": "why is",
    "let's": "let us",
    "'re": " are",
    "'s": " is",
    "'d": " would",
    "'ll": " will",
    "'ve": " have",
    "n't": " not",
    "nt":" not",
}

## Cleaning Text¶

In [13]:
def get_wordnet_pos(treebank_tag):

    # Part_Of_Speech
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def clean_text(text, chat_words, contractions):

    # @UserName:
    text = re.sub(r'@\w+', '', text)

    # Timestamps
    text = re.sub(r'\d{2,4}[-/]\d{1,2}[-/]\d{1,2} \d{1,2}:\d{2}', '', text)
    text = re.sub(r'\d{1,2}:\d{2}', '', text)

    # HTML Tags:
    pattern = re.compile('<.*?>')
    text = pattern.sub(r'', text)

    # Links:
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'www\S+', '', text)

    # All text to Lowercase:
    text = text.lower()

    # Replace By Contractions:
    for k, v in contractions.items():
        pattern = r'\b' + re.escape(k.lower()) + r'\b'
        text = re.sub(pattern, v.lower(), text)

    # Replace By Abbreviations:
    for k, v in chat_words.items():
        pattern = r'\b' + re.escape(k.lower()) + r'\b'
        text = re.sub(pattern, v.lower(), text)

    # Remove Punctuation:
    for char in string.punctuation:
        text = text.replace(char,'')

    # Text Only:
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # # TextBlob
    # blob = TextBlob(text)
    # text = str(blob.correct())

    # Tokenization:
    words = nltk.word_tokenize(text)

    # Lemmatization:
    lemmatizer = WordNetLemmatizer()
    pos_tags = nltk.pos_tag(words)
    words = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tags]

    # Ending with "ing" ############################
    # new_words = []
    # for w in words:
    #     if w.endswith("ing") and len(w) > 4:
    #         new_words.append(w[:-3])
    #     else:
    #         new_words.append(w)
    # words = new_words

    # Remove stop words:
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in words if word not in stop_words]
    text = ' '.join(tokens)

    return text

In [14]:
# Applying Function:
df.loc[:, "question"] = df["question"].apply(lambda x: clean_text(x, chat_words, contractions))
df.loc[:, "answer"] = df["answer"].apply(lambda x: clean_text(x, chat_words, contractions))

In [15]:
df["question"], df["answer"]

(0                 well think would start pronunciation okay
 1                          hacking gagging spit part please
 2                                             ask cute name
 3                                 fault proper introduction
 4                             gosh could find kat boyfriend
                                 ...                        
 139404                               well one one keep look
 139405                 choose target men right watch marker
 139406    colonel durnford william vereker hear seek off...
 139407                                     order mr vereker
 139408                        take sikali main column river
 Name: question, Length: 138699, dtype: object,
 0                         hacking gagging spit part please
 1              okay bout try french cuisine saturday night
 2                                                   forget
 3                                                  cameron
 4                                       

In [16]:
df["text"] = "User: " + df["question"] + "\nAssistant: " + df["answer"]
texts = df["text"].tolist()

C:\Users\Nitro\AppData\Local\Temp\ipykernel_18512\3058211759.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"] = "User: " + df["question"] + "\nAssistant: " + df["answer"]


In [17]:
df.head()

,question,answer,text
0,well think would start pronunciation okay,hacking gagging spit part please,User: well think would start pronunciation oka...
1,hacking gagging spit part please,okay bout try french cuisine saturday night,User: hacking gagging spit part please\nAssist...
2,ask cute name,forget,User: ask cute name\nAssistant: forget
3,fault proper introduction,cameron,User: fault proper introduction\nAssistant: ca...
4,gosh could find kat boyfriend,let see,User: gosh could find kat boyfriend\nAssistant...


## Training

In [18]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
tokenizer.pad_token = tokenizer.eos_token

In [19]:
from torch.utils.data import Dataset

class QADataset(Dataset):
    def __init__(self, texts, tokenizer, max_length=256):
        self.input_ids = []
        self.attn_masks = []

        for t in texts:
            enc = tokenizer(
                t,
                truncation=True,
                max_length=max_length,
                padding="max_length",
                return_tensors="pt"
            )
            self.input_ids.append(enc["input_ids"][0])
            self.attn_masks.append(enc["attention_mask"][0])

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attn_masks[idx],
            "labels": self.input_ids[idx],
        }

In [20]:
dataset = QADataset(texts, tokenizer)

In [21]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "openai-community/gpt2"
)

# Match padding token
model.config.pad_token_id = model.config.eos_token_id

In [22]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./gpt2-finetuned-chatbot",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_steps=50,
    save_steps=2000,
    save_total_limit=2,
    fp16=True,  # if GPU supports it
)

In [27]:
from transformers import GPT2LMHeadModel

model.save_pretrained("chatbot_model")


In [28]:
tokenizer.save_pretrained("chatbot_model")


('chatbot_model\\tokenizer_config.json',
 'chatbot_model\\special_tokens_map.json',
 'chatbot_model\\vocab.json',
 'chatbot_model\\merges.txt',
 'chatbot_model\\added_tokens.json',
 'chatbot_model\\tokenizer.json')